In [1]:
import json

In [2]:
models_paths = {
    "3" : "./finetuned_model/checkpoint-435",
    "5" : "./finetuned_model/checkpoint-725",
    "10" : "./finetuned_model/checkpoint-1450",
    "15" : "./finetuned_model/checkpoint-2175",
    "20" : "./finetuned_model/checkpoint-2900",
    "30" : "./finetuned_model/checkpoint-4350",
    "40" : "./finetuned_model/checkpoint-5800",
    "50" : "./finetuned_model/checkpoint-7105"
}

llama_model_paths = {
 '4': './finetuned_llama3-2-3B_model/checkpoint-148',
 '5': './finetuned_llama3-2-3B_model/checkpoint-185',
 '6': './finetuned_llama3-2-3B_model/checkpoint-222',
 '7': './finetuned_llama3-2-3B_model/checkpoint-259',
 '8': './finetuned_llama3-2-3B_model/checkpoint-296',
 '9': './finetuned_llama3-2-3B_model/checkpoint-333',
 '10': './finetuned_llama3-2-3B_model/checkpoint-370',
 '11': './finetuned_llama3-2-3B_model/checkpoint-407',
 '12': './finetuned_llama3-2-3B_model/checkpoint-444',
 '13': './finetuned_llama3-2-3B_model/checkpoint-481',
 '14': './finetuned_llama3-2-3B_model/checkpoint-518',
 '15': './finetuned_llama3-2-3B_model/checkpoint-540'}

# Dataset

In [3]:
filename = "/home/raza/Documents/onboarding-task/llama-3-finetuning-data.jsonl"

In [4]:
data = []
with open(filename, 'r', encoding='utf-8') as f:
  for line in f:
    data.append(json.loads(line.strip()))


In [5]:
len(data)

703

In [6]:
from datasets import Dataset

dataset = Dataset.from_list(data)

train_test_data = dataset.train_test_split(test_size=63, seed=42)
train_val_data = train_test_data["train"].train_test_split(test_size=63, seed=42)

# Access the train, validation, and test datasets
train_dataset = train_val_data["train"]
eval_dataset = train_val_data["test"]
test_dataset = train_test_data["test"]

# Print the number of examples in each set
print(f"Train dataset: {len(train_dataset)} examples")
print(f"Validation dataset: {len(eval_dataset)} examples")
print(f"Test dataset: {len(test_dataset)} examples")



Train dataset: 577 examples
Validation dataset: 63 examples
Test dataset: 63 examples


In [7]:
test_dataset[13]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.<|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\nDefinition of a Functional Area:\n- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.\n\nDefinition of Management Level:\n- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.\n- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."\n\nDefinition of a Job Title:\n- A

# Loading Libraries

In [8]:
# base_model = "/home/raza/Downloads/Phi-3-mini-4k-instruct"
# peft_model_name = models_paths["10"] 

peft_model_name = llama_model_paths["10"] 
base_model = "/home/raza/Downloads/meta-llama-3-2-3B-Instruct"

In [9]:
from transformers import BitsAndBytesConfig, AutoProcessor, AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

# processor = AutoProcessor.from_pretrained(base_model)
# processor.padding_side = 'right'


processor = AutoTokenizer.from_pretrained(base_model)
processor.padding_side = 'right'

# Set the padding token to be the same as eos_token if it's not defined
if processor.pad_token is None:
    processor.pad_token = processor.eos_token  # or you can set a custom token if needed


In [11]:

model = AutoModelForCausalLM.from_pretrained(
          base_model,
          trust_remote_code=True,
          attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
          torch_dtype="auto",
          use_cache=False, # set to False as we're going to use gradient checkpointing
          device_map=device_map,
          quantization_config=quantization_config,
)

peft_model = PeftModel.from_pretrained(
    model,
    peft_model_name
)

peft_model.eval()



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [12]:
def inference_function(model, max_new_tokens, input_text: str):
    """
    Simple inference function to generate text based on an input prompt.

    Args:
    - input_text (str): The input text to be processed by the model.
    - max_length (int): The maximum length of the generated text.

    Returns:
    - str: The generated text.
    """
    # Process the input text
    inputs = processor(input_text, return_tensors="pt", padding=True).to(model.device)
    # Generate output from the model
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,  # Enable sampling for more diverse outputs
        temperature=0.1,
        top_p=0.9
    )

    # Decode the output tokens into text
    generated_text = processor.decode(output[0], skip_special_tokens=True)  # Get the first sequence

    return generated_text

# Example usage
input_text = "Donald Trump is "
generated_output = inference_function(model=model, input_text=input_text, max_new_tokens=8)
print(f"Generated Output: {generated_output}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Output: Donald Trump is 75 years old, born on June 


# Util Functions

In [13]:
import json

def parse_string(input_string):
    # Define the markers and unwanted "Output:" part
    marker = "<|assistant|>"
    output_label = "Output:"
    end_marker = "<|end|>"  # Marker that may appear at the end of the string
    
    # Find the index of the main marker
    marker_index = input_string.find(marker)

    # If the marker is found, split the string accordingly
    if marker_index != -1:
        prompt = input_string[:marker_index].strip() + marker + """{ "management_level" : """
        ground_truth = input_string[marker_index + len(marker):].strip()
        
        # Remove the "Output:" label if it's present at the beginning of the ground truth
        if ground_truth.startswith(output_label):
            ground_truth = ground_truth[len(output_label):].strip()
        
        # Remove the "<|end|>" token or other similar tokens from the ground truth
        ground_truth = ground_truth.replace(end_marker, "").strip()

        # Convert single quotes to double quotes for valid JSON parsing
        ground_truth = ground_truth.replace("'", "\"")

        # Try parsing the ground truth into a dictionary
        try:
            ground_truth_dict = json.loads(ground_truth)  # Assumes ground truth is a valid JSON string
        except json.JSONDecodeError:
            ground_truth_dict = None  # If it's not valid JSON, return None
            
        return {
            "prompt": prompt,
            "ground_truth": ground_truth_dict
        }
    else:
        # If the marker is not found, return the entire input as prompt, and None as ground_truth
        return {
            "prompt": input_string.strip(),
            "ground_truth": None
        }


parsed_result = parse_string(test_dataset[13]["text"])

print(json.dumps(parsed_result, indent=2))


{
  "prompt": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.<|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\nDefinition of a Functional Area:\n- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.\n\nDefinition of Management Level:\n- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.\n- Management levels include: \"Board of Directors,\" \"CSuite and President,\" \"Executive and Sr. VP,\" \"General Manager,\" \"VP,\" \"Director,\" \"Manager,\" \"Senior (Individual Contributor),\" \"Mid (Individual Contributor),\" and \"Junior.\"\

In [14]:
import re

def extract_query(input_str):
    # Define the pattern to capture the last "Query:" and corresponding "Output:"
    pattern = r'Use knowledge base for educated guesses about classifications\s*Query:\s*(.*?)\s*Output:'
    
    # Search for the pattern
    match = re.search(pattern, input_str, re.DOTALL)
    
    if match:
        # Extract the query text (the first group corresponds to the query part)
        query = match.group(1).strip()
        return eval(query)
    else:
        return None
    
extract_query(test_dataset[13]["text"])

'VPs of development or executives in real estate?'

In [15]:
import json
import re

def response_parser(response_str):
    # Define a regular expression pattern to capture all JSON objects in the string
    json_pattern = r'(\{.*?\})'
    
    # Find all matches of the JSON objects in the string
    matches = re.findall(json_pattern, response_str.strip(), re.DOTALL)
    
    if matches:
        # Get the last match, which should be the most recent JSON object
        last_json_str = matches[-1]
        
        try:
            # Parse the last matched JSON string
            parsed_response = json.loads(last_json_str)
            response_list = []
            for key,value in parsed_response.items():
                response_list.append(value)

            if len(response_list) == 2:
                parsed_response = {'management_levels': response_list[0], 'titles': response_list[1]}
            else:
                parsed_response = response_list
            
            return parsed_response
        except json.JSONDecodeError:
            print("Error: The last JSON in the response string is not valid.")
            return None
    else:
        print("Error: No valid JSON found in the response.")
        return None


In [16]:
import pandas as pd
from IPython.display import display

def display_comparison(comparison_list):
    # Helper function to safely get key from dictionary or return a default value
    def safe_get(entry, field, subfield=None, default="N/A"):
        if entry.get(field) is None:  # Check if the field itself is None
            return default
        if subfield:  # If we need to access a subfield (e.g., 'label', 'confidence')
            return entry.get(field, {}).get(subfield, default)
        return entry.get(field, default)
    
    # Extract relevant information from Query, Ground Truth, Prediction, and Exact Match
    query_data = [entry.get('Query', 'N/A') for entry in comparison_list]
    
    # Extract 'Ground Truth' data, handling missing keys or NoneType
    gt_data = [
        f"{safe_get(entry, 'Ground Truth')}"
        for entry in comparison_list
    ]
    
    # Extract 'Prediction' data, handling missing keys or NoneType
    pred_data = [
        f"{safe_get(entry, 'Prediction')}"
        for entry in comparison_list
    ]


    
    # Extract 'Exact Match' information (assuming it's a boolean)
    exact_match_data = [
        "Yes" if entry.get('Exact Match', False) else "No"  # Default to False if 'Exact Match' is missing or None
        for entry in comparison_list
    ]


    
    # Create a DataFrame with the new information
    df = pd.DataFrame({
        'Query': query_data,
        'Ground Truth (GT)': gt_data,
        'Prediction (Pred)': pred_data,
        'Exact Match': exact_match_data
        
    })
    
    # Display the DataFrame as a neat table
    display(df)
    return df


In [17]:
ground_truth_list = [{'query': 'From the latest report of Genentech, tell me the key highlights of the company.',
  'GT': {'management_levels': [], 'titles': []}},
 {'query': 'Director of Human-AI Collaboration',
  'GT': {'management_levels': [],
   'titles': ['Director of Human-AI Collaboration']}},
 {'query': 'Identify junior roles in cloud services.',
  'GT': {'management_levels': ['Junior'], 'titles': []}},
 {'query': 'List all senior engineers in robotics innovation labs.',
  'GT': {'management_levels': ['Senior (Individual Contributor)'],
   'titles': ['Engineer']}},
 {'query': 'Director Mental Health Programs',
  'GT': {'management_levels': [],
   'titles': ['Director of Mental Health Programs']}},
 {'query': 'Leaders of top energy companies in the US.',
  'GT': {'management_levels': ['CSuite and President'], 'titles': []}},
 {'query': 'VP Planetary Science',
  'GT': {'management_levels': [],
   'titles': ['VP of Planetary Science',
    'Vice President of Planetary Science']}},
 {'query': 'VP Construction Innovation',
  'GT': {'management_levels': [],
   'titles': ['VP of Construction Innovation',
    'Vice President of Construction Innovation']}},
 {'query': 'Directors or managers leading innovation projects in IT.',
  'GT': {'management_levels': ['Director', 'Manager'], 'titles': []}},
 {'query': 'Chief Archaeologist',
  'GT': {'management_levels': [], 'titles': ['Chief Archaeologist']}},
 {'query': 'VP Marine Innovation',
  'GT': {'management_levels': [],
   'titles': ['VP of Marine Innovation',
    'Vice President of Marine Innovation']}},
 {'query': 'Heads of marketing for social media platforms.',
  'GT': {'management_levels': [], 'titles': ['Head of Marketing']}},
 {'query': 'Chief Data Ethics Officer',
  'GT': {'management_levels': [], 'titles': ['Chief Data Ethics Officer']}},
 {'query': 'VPs of development or executives in real estate?',
  'GT': {'management_levels': ['Executive and Sr. VP'],
   'titles': ['VP of Development', 'Vice President of Development']}},
 {'query': 'Leaders of government tech initiatives in Asia.',
  'GT': {'management_levels': ['CSuite and President'], 'titles': []}},
 {'query': 'VP Cultural Communications',
  'GT': {'management_levels': [],
   'titles': ['VP of Cultural Communications',
    'Vice President of Cultural Communications']}},
 {'query': 'Director Neural Implants',
  'GT': {'management_levels': [], 'titles': ['Director of Neural Implants']}},
 {'query': 'VP CRISPR Applications',
  'GT': {'management_levels': [],
   'titles': ['VP of CRISPR Applications',
    'Vice President of CRISPR Applications']}},
 {'query': 'Find all CTOs, CFOs, and CMOs in AI startups.',
  'GT': {'management_levels': [],
   'titles': ['Chief Technology Officer',
    'CTO',
    'Chief Financial Officer',
    'CFO',
    'Chief Marketing Officer',
    'CMO']}},
 {'query': 'Chief Human Potential Officer',
  'GT': {'management_levels': [],
   'titles': ['Chief Human Potential Officer']}},
 {'query': 'Division chiefs in government',
  'GT': {'management_levels': [], 'titles': ['Division Chief']}},
 {'query': 'Executives in pharma and leaders in biotech.',
  'GT': {'management_levels': ['CSuite and President'],
   'titles': ['Executive']}},
 {'query': 'List board members of top banks in Europe.',
  'GT': {'management_levels': ['Board of Directors'], 'titles': []}},
 {'query': 'Chief Digital Transformation Officer',
  'GT': {'management_levels': [],
   'titles': ['Chief Digital Transformation Officer']}},
 {'query': 'Director Climate Studies',
  'GT': {'management_levels': [], 'titles': ['Director of Climate Studies']}},
 {'query': 'VP Engineering & Architecture positions',
  'GT': {'management_levels': [],
   'titles': ['VP of Engineering & Architecture',
    'Vice President of Engineering & Architecture']}},
 {'query': 'Find senior strategists in public policy organizations.',
  'GT': {'management_levels': ['Senior (Individual Contributor)'],
   'titles': ['Strategist']}},
 {'query': 'Director of Quantum Initiatives',
  'GT': {'management_levels': [],
   'titles': ['Director of Quantum Initiatives']}},
 {'query': 'Find research assistants specializing in quantum computing.',
  'GT': {'management_levels': [], 'titles': ['Research Assistant']}},
 {'query': 'Head of AI/ML Research',
  'GT': {'management_levels': [], 'titles': ['Head of AI/ML Research']}},
 {'query': 'List all leaders in the construction sector.',
  'GT': {'management_levels': ['CSuite and President'], 'titles': []}},
 {'query': 'Leaders responsible for diversity and inclusion initiatives.',
  'GT': {'management_levels': ['CSuite and President'], 'titles': []}},
 {'query': 'List interns contributing to social media campaigns.',
  'GT': {'management_levels': [], 'titles': ['Intern']}},
 {'query': 'Chief Mindfulness Officer',
  'GT': {'management_levels': [], 'titles': ['Chief Mindfulness Officer']}},
 {'query': 'Chief Future Officer',
  'GT': {'management_levels': [], 'titles': ['Chief Future Officer']}},
 {'query': 'Who leads the logistics team in FMCG companies?',
  'GT': {'management_levels': [], 'titles': ['Leader of Logistics']}},
 {'query': 'Director Infrastructure Planning',
  'GT': {'management_levels': [],
   'titles': ['Director of Infrastructure Planning']}},
 {'query': 'VP Ancient Studies',
  'GT': {'management_levels': [],
   'titles': ['VP of Ancient Studies', 'Vice President of Ancient Studies']}},
 {'query': 'Chief Marketing & Growth Officer',
  'GT': {'management_levels': [],
   'titles': ['Chief Marketing & Growth Officer']}},
 {'query': 'Find assistant professors specializing in data science.',
  'GT': {'management_levels': [], 'titles': ['Assistant Professor']}},
 {'query': 'Find senior managers in manufacturing.',
  'GT': {'management_levels': ['Senior (Individual Contributor)', 'Manager'],
   'titles': []}},
 {'query': 'Executives in renewable energy',
  'GT': {'management_levels': ['Executive and Sr. VP'], 'titles': []}},
 {'query': 'Regional Director and Territory Manager positions',
  'GT': {'management_levels': ['Director', 'Manager'],
   'titles': ['Regional Director', 'Territory Manager']}},
 {'query': 'mid-level managers and senior managers in retail',
  'GT': {'management_levels': ['Manager',
    'Senior (Individual Contributor)',
    'Mid (Individual Contributor)'],
   'titles': []}},
 {'query': 'List the interns involved in environmental policy research.',
  'GT': {'management_levels': ['Junior'], 'titles': []}},
 {'query': 'Head of Growth & VP Growth positions',
  'GT': {'management_levels': ['VP'],
   'titles': ['Head of Growth', 'Vice President of Growth', 'VP of Growth']}},
 {'query': 'Chief Experience Officer positions',
  'GT': {'management_levels': [], 'titles': ['Chief Experience Officer']}},
 {'query': 'Who leads the product innovation teams in tech startups?',
  'GT': {'management_levels': ['CSuite and President'], 'titles': []}},
 {'query': 'Executives at global pharma companies.',
  'GT': {'management_levels': ['Executive and Sr. VP'], 'titles': []}},
 {'query': 'Chief Marketing Officer or VP Marketing roles in FAANG',
  'GT': {'management_levels': ['VP'],
   'titles': ['Chief Marketing Officer',
    'CMO',
    'VP of Marketing',
    'Vice President of Marketing']}},
 {'query': 'VP Augmented Intelligence',
  'GT': {'management_levels': [],
   'titles': ['VP of Augmented Intelligence',
    'Vice President of Augmented Intelligence']}},
 {'query': 'University President and Provost',
  'GT': {'management_levels': ['CSuite and President'],
   'titles': ['Provost']}},
 {'query': 'List junior associates in investment banking teams.',
  'GT': {'management_levels': ['Junior'],
   'titles': ['Associate in Investment Banking']}},
 {'query': 'Chief Epidemiologist',
  'GT': {'management_levels': [], 'titles': ['Chief Epidemiologist']}},
 {'query': 'sr. vp global operations',
  'GT': {'management_levels': ['Executive and Sr. VP'],
   'titles': ['Senior Vice President of Global Operations',
    'Sr. VP of Global Operations']}},
 {'query': 'Find mid-level contributors in healthcare.',
  'GT': {'management_levels': ['Mid (Individual Contributor)'], 'titles': []}},
 {'query': 'List leaders or directors in environmental projects.',
  'GT': {'management_levels': ['CSuite and President', 'Director'],
   'titles': []}},
 {'query': 'Identify senior executives in fintech.',
  'GT': {'management_levels': ['Executive and Sr. VP'], 'titles': []}},
 {'query': 'Executives in fintech startups focusing on compliance.',
  'GT': {'management_levels': ['Executive and Sr. VP'], 'titles': []}},
 {'query': 'Chief Alternative Energy Officer',
  'GT': {'management_levels': [],
   'titles': ['Chief Alternative Energy Officer']}},
 {'query': 'List board members in the banking sector.',
  'GT': {'management_levels': ['Board of Directors'], 'titles': []}},
 {'query': 'VP People Operations & Culture',
  'GT': {'management_levels': [],
   'titles': ['VP of People Operations & Culture',
    'Vice President of People Operations & Culture']}},
 {'query': 'Director Quantum Biology',
  'GT': {'management_levels': [], 'titles': ['Director of Quantum Biology']}}]

# Generating Results

In [18]:
scores = []

In [19]:
unparsed_responses = []

In [20]:

## Loading Checkpoint
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        trust_remote_code=True,
        attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
        torch_dtype="auto",
        use_cache=False, # set to False as we're going to use gradient checkpointing
        device_map=device_map,
        quantization_config=quantization_config,
)

results_list = []
em_count = 0
responses_list = []

for ground_truth, item in zip(ground_truth_list, test_dataset):
    parsed_data = parse_string(item["text"])
    query = extract_query(item["text"])
    prompt = parsed_data["prompt"]
    response = inference_function(model=model, input_text=prompt, max_new_tokens=1024)
    responses_list.append(response)
    parsed_response = response_parser(response)
    if ground_truth["query"] == query:
        exact_match = ground_truth["GT"] == parsed_response
    else:
        exact_match = "wrong query"

    if exact_match == True:
        em_count+=1
    results_list.append({"Query": query,  "Ground Truth" : ground_truth["GT"],"Prediction": parsed_response, "Exact Match": exact_match})
unparsed_responses.append({
    f"base": responses_list
})
print("Exact Match Count: ",em_count)
print("Exact Match Accuracy: ", em_count/len(test_dataset) * 100)

scores.append({"epoch_number": "base","em_count" : em_count, "em_percentage" : em_count/len(test_dataset) * 100})

df = display_comparison(results_list)
df.to_csv(f"/home/raza/Documents/onboarding-task/llama-3-results/base-results.csv")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.
Exact Match Count:  1
Exact Match Accuracy:  1.5873015873015872


,Query,Ground Truth (GT),Prediction (Pred),Exact Match
0,"From the latest report of Genentech, tell me t...","{'management_levels': [], 'titles': []}",N/A,No
1,Director of Human-AI Collaboration,"{'management_levels': [], 'titles': ['Director...",N/A,No
2,Identify junior roles in cloud services.,"{'management_levels': ['Junior'], 'titles': []}",N/A,No
3,List all senior engineers in robotics innovati...,{'management_levels': ['Senior (Individual Con...,N/A,No
4,Director Mental Health Programs,"{'management_levels': [], 'titles': ['Director...",N/A,No
...,...,...,...,...
58,Executives in fintech startups focusing on com...,{'management_levels': ['Executive and Sr. VP']...,N/A,No
59,Chief Alternative Energy Officer,"{'management_levels': [], 'titles': ['Chief Al...","{'management_levels': [], 'titles': ['Chief Al...",No
60,List board members in the banking sector.,"{'management_levels': ['Board of Directors'], ...",N/A,No
61,VP People Operations & Culture,"{'management_levels': [], 'titles': ['VP of Pe...",N/A,No


In [21]:
with open("llama-3-results/baseunparsed.json", "w") as f:
    json.dump(unparsed_responses, f, indent=2)

In [22]:
for key, value in llama_model_paths.items():
    count = 0
    print('-'*47)
    epoch_number = key
    peft_model_name = value
    print(epoch_number, peft_model_name)

    ## Loading Checkpoint
    model = AutoModelForCausalLM.from_pretrained(
          base_model,
          trust_remote_code=True,
          attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
          torch_dtype="auto",
          use_cache=False, # set to False as we're going to use gradient checkpointing
          device_map=device_map,
          quantization_config=quantization_config,
    )

    peft_model = PeftModel.from_pretrained(
        model=model,
        model_id=peft_model_name,
    )

    peft_model.eval()

    results_list = []
    responses_list = []
    em_count = 0

    

    for ground_truth, item in zip(ground_truth_list, test_dataset):

        parsed_data = parse_string(item["text"])
        query = extract_query(item["text"])
        prompt = parsed_data["prompt"]
        response = inference_function(model=peft_model, input_text=prompt, max_new_tokens=1024)
        responses_list.append(response)
        parsed_response = response_parser(response)
        if ground_truth["query"] == query:
            exact_match = ground_truth["GT"] == parsed_response
        else:
            exact_match = "wrong query"
        if exact_match == True:
            em_count+=1
        results_list.append({"Query": query, "Ground Truth": ground_truth["GT"], "Prediction": parsed_response, "Exact Match": exact_match})
    unparsed_responses.append({
        f"{epoch_number} epoch" : responses_list
    })
    print("Exact Match Count: ",em_count)
    print("Exact Match Accuracy: ", em_count/len(test_dataset) * 100)
    scores.append({"epoch_number": epoch_number,"em_count" : em_count, "em_percentage" : em_count/len(test_dataset) * 100})

    df = display_comparison(results_list)
    df.to_csv(f"/home/raza/Documents/onboarding-task/llama-3-results/{epoch_number}-epoch-results.csv")
    with open(f"llama-3-results/{epoch_number}unparsed.json", "w") as f:
        json.dump(unparsed_responses, f, indent=2)

    torch.cuda.empty_cache()



    
    



-----------------------------------------------
4 ./finetuned_llama3-2-3B_model/checkpoint-148


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error: The last JSON in the response string is not valid.
Error: The last JSON in the response string is not valid.
Exact Match Count:  0
Exact Match Accuracy:  0.0


,Query,Ground Truth (GT),Prediction (Pred),Exact Match
0,"From the latest report of Genentech, tell me t...","{'management_levels': [], 'titles': []}",N/A,No
1,Director of Human-AI Collaboration,"{'management_levels': [], 'titles': ['Director...",N/A,No
2,Identify junior roles in cloud services.,"{'management_levels': ['Junior'], 'titles': []}",N/A,No
3,List all senior engineers in robotics innovati...,{'management_levels': ['Senior (Individual Con...,N/A,No
4,Director Mental Health Programs,"{'management_levels': [], 'titles': ['Director...",N/A,No
...,...,...,...,...
58,Executives in fintech startups focusing on com...,{'management_levels': ['Executive and Sr. VP']...,N/A,No
59,Chief Alternative Energy Officer,"{'management_levels': [], 'titles': ['Chief Al...",N/A,No
60,List board members in the banking sector.,"{'management_levels': ['Board of Directors'], ...",N/A,No
61,VP People Operations & Culture,"{'management_levels': [], 'titles': ['VP of Pe...",N/A,No


In [23]:
scores

[{'epoch_number': 'base', 'em_count': 1, 'em_percentage': 1.5873015873015872},
 {'epoch_number': '4', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '5', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '6', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '7', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '8', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '9', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '10', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '11', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '12', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '13', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '14', 'em_count': 0, 'em_percentage': 0.0},
 {'epoch_number': '15', 'em_count': 0, 'em_percentage': 0.0}]

In [ ]:
# T1
[{'epoch_number': 'base', 'em_count': 3, 'em_percentage': 4.761904761904762},
 {'epoch_number': '3', 'em_count': 7, 'em_percentage': 11.11111111111111},
 {'epoch_number': '5', 'em_count': 8, 'em_percentage': 12.698412698412698},
 {'epoch_number': '10', 'em_count': 14, 'em_percentage': 22.22222222222222},
 {'epoch_number': '15', 'em_count': 10, 'em_percentage': 15.873015873015872},
 {'epoch_number': '20', 'em_count': 10, 'em_percentage': 15.873015873015872},
 {'epoch_number': '30', 'em_count': 11, 'em_percentage': 17.46031746031746},
 {'epoch_number': '40', 'em_count': 9, 'em_percentage': 14.285714285714285},
 {'epoch_number': '50', 'em_count': 13, 'em_percentage': 20.634920634920633}]

In [27]:
results_df = pd.read_csv("/home/raza/Documents/onboarding-task/phi-3-results/10-epoch-results.csv")

In [28]:
results_df

,Unnamed: 0,Query,Ground Truth (GT),Prediction (Pred),Exact Match
0,0,"From the latest report of Genentech, tell me t...","{'management_levels': [], 'titles': []}","{'management_level': [], 'title': ['Genentech']}",No
1,1,Director of Human-AI Collaboration,"{'management_levels': [], 'titles': ['Director...","{'management_level': [], 'title': ['Director o...",No
2,2,Identify junior roles in cloud services.,"{'management_levels': ['Junior'], 'titles': []}","{'management_level': ['Junior'], 'title': ['Cl...",No
3,3,List all senior engineers in robotics innovati...,{'management_levels': ['Senior (Individual Con...,{'management_level': ['Senior (Individual Cont...,No
4,4,Director Mental Health Programs,"{'management_levels': [], 'titles': ['Director...","{'management_level': ['Manager'], 'title': ['D...",No
...,...,...,...,...,...
58,58,Executives in fintech startups focusing on com...,{'management_levels': ['Executive and Sr. VP']...,"{'management_level': ['Executive and Sr. VP'],...",No
59,59,Chief Alternative Energy Officer,"{'management_levels': [], 'titles': ['Chief Al...","{'management_level': ['Executive and Sr. VP'],...",No
60,60,List board members in the banking sector.,"{'management_levels': ['Board of Directors'], ...","{'management_level': ['Board of Directors'], '...",No
61,61,VP People Operations & Culture,"{'management_levels': [], 'titles': ['VP of Pe...","{'management_level': ['VP'], 'title': ['VP of ...",No
